# Upload Block VQE

I will use this file to write the runtime program and then to upload it to the server.

The structure of the file will be as follows:

1) define the circuits

    a) diagonal
    b) off diagonal
    
2) define the energy

    a) measure_pauli
    b) energy on block
    c) energy off block
    d) total energy
    
3) define the SPSA optimization

In [16]:
%%writefile runtime_block_vqe.py

####### Circuits ########
from qiskit import QuantumCircuit, QuantumRegister,ClassicalRegister, execute
from qiskit import transpile

##on-block##

def Ui(cir_i,phi_i):  
    Q = len(phi_i)
    for q in range(0,Q):
        cir_i.ry(phi_i[q],q)
    for q in range(0,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    for q in range(1,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    
    return cir_i

def U(cir,phi_b):
    T = len(phi_b)
    for t in range(T):
        cir = Ui(cir,phi_b[t])
    return cir

##of-block##
def Ui_off(cir_i,phi_a, phi_b):  
    Q = len(phi_a)
    for q in range(0,Q):
        cir_i.ry(phi_a[q],q)
        cir_i.cx(Q,q)
        cir_i.ry(-(phi_b[q]-phi_a[q])/2,q)
        cir_i.cx(Q,q)
        cir_i.ry((phi_b[q]-phi_a[q])/2,q)
    for q in range(0,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    for q in range(1,Q,4):
        cir_i.cx(q,q+1)
        if q + 3 < Q:
            cir_i.cx(q+3,q+2)
    
    return cir_i

def U_off(cir,phi_a,phi_b):
    T = len(phi_b)
    Q = len(phi_b[0])
    cir.h(Q)
    for t in range(T):
        cir = Ui_off(cir,phi_a[t],phi_b[t])
    return cir



####### Calculating Energy ########
from qiskit import Aer
from qiskit.visualization import *
from qiskit import quantum_info as qi


##measure paulis##
def measure_pauli(backend,p_label,psi0,method = 'simple'):
    
    #Simple method of calculation####
    if method == 'simple':
        wave0 = qi.Statevector.from_instruction(psi0)
        Op = qi.Operator.from_label(p_label)
        return np.dot(np.conjugate(wave0),np.dot(Op,wave0))
    #################################
        
    #apply rotations#################
    psi = copy.deepcopy(psi0)
    pauli_qs = []
    Z_label = ''
    Q = len(p_label)
    for q,p in enumerate(p_label):
        if p == 'X':
            psi.ry(-np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Y':
            psi.rx(np.pi/2,Q-1-q)
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'Z':
            pauli_qs.append(1)
            Z_label += 'Z'
        if p == 'I':
            pauli_qs.append(0)
            Z_label += 'I'
    ################################
    
    #Using matrix multiplication####
    if method == 'matrix':
        # add results
        wave = qi.Statevector.from_instruction(psi)
        r = wave.probabilities_dict()
        z_measure = 0
        for key in list(r.keys()):
            n = 0
            for q in range(len(key)):
                if key[q] == '1' and pauli_qs[q] == 1:
                    n += 1
            z_measure += (-1)**n * r[key] 
        return z_measure
    #################################
    
    #Using the qasm simulator########
    if method == "simulator":
        sim = Aer.get_backend("qasm_simulator")
        psi.measure(psi.qubits,psi.clbits)
        r = execute(psi, backend = sim).result().get_counts()
        z_measure = 0
        total = 0
        for key in list(r.keys()):
            n = 0
            for q in range(len(key)):
                if key[q] == '1' and pauli_qs[q] == 1:
                    n += 1
            z_measure += (-1)**n * r[key] 
            total += r[key]
        return z_measure/total
    ###################################
    
    #Using the quantum backend#########
    if method == "quantum":
        psi.measure(psi.qubits,psi.clbits)
        job = backend.run(transpile(psi, backend), meas_level=2, shots=8192) 
        #print("job id: ",job.job_id())
        #print(job_monitor(job))
        r = job.result().get_counts()
        z_measure = 0
        total = 0
        for key in list(r.keys()):
            n = 0
            for q in range(len(key)):
                if key[q] == '1' and pauli_qs[q] == 1:
                    n += 1
            z_measure += (-1)**n * r[key] 
            total += r[key]

        z_measure/total
        return z_measure/total
    ###################################
    
    raise NameError(method + ' is not a recognized method')
    return method + ' is not a recognized method'

##energy on block##
def E_on_block(backend,phi_b,block,method = 'simple'):
    E = 0
    Q = len(phi_b[0])
    qr = QuantumRegister(Q)
    cr = ClassicalRegister(Q)
    cir = QuantumCircuit(qr , cr)
    psi0 = U(cir,phi_b)
    for p in block:
        psi = copy.deepcopy(psi0)
        w = p.coeff
        p_label = p.primitive.to_label()
        E_p = measure_pauli(backend,p_label,psi,method = method)
        E += w*E_p
    return E

##energy off block##
def E_off_block(backend,phi_a,phi_b,block,method = 'simple'):
    E = 0
    Q = len(phi_b[0])
    qr = QuantumRegister(Q+1)
    cr = ClassicalRegister(Q+1)
    cir = QuantumCircuit(qr , cr)
    psi0 = U_off(cir,phi_a,phi_b)
    for p in block:
        psi = copy.deepcopy(psi0)
        w = p.coeff
        p_label = p.primitive.to_label()
        E_px = measure_pauli(backend,'X' + p_label,psi,method = method)
        E_py = measure_pauli(backend,'Y' + p_label,psi,method = method)
        E += w*(E_px + 1j*E_py)/2
    return E

##total energy##
def find_E(backend,alpha,phi,blocks,method = 'simple'):
    E = 0
    for key in list(blocks.keys()):
        if key[0] == key[2]:
            E += alpha[int(key[0])]*alpha[int(key[2])]*E_on_block(backend,phi[int(key[0])],blocks[key],method = method)
        else:
            E += 2*alpha[int(key[0])]*alpha[int(key[2])]*E_off_block(backend,phi[int(key[0])],phi[int(key[2])],blocks[key],method = method)
    return E



####### Optimization ########
import copy
import numpy as np

beta = 0.201
A = 10
a = 0.05
gamma = 0.101
c = 0.4

beta_a = 0.201
A_a = 10
a_a = 0.05
gamma_a = 0.101
c_a = 0.4



def SPSA(backend, user_messenger, k_max, phi, alpha, blocks, method = 'simple',hold = False, seeded = False):
    #Initalization
    if seeded: 
        user_messenger.publish({'seeding is on':seeded})
        np.random.seed(0)
    k = 0
    phi_k = np.array(phi)
    alpha_k = np.array(alpha)
    bL = len(phi_k)
    T = len(phi_k[0])
    Q = len(phi_k[0][0])
    E_l = []
    hold_l = []
    
    #Begin Iterations
    for k in range(k_max):
        #Update c and a
        a_k = a/((A + k + 1)**beta)
        c_k = c/((k + 1)**gamma)
        a_ak = a_a/((A_a + k + 1)**beta_a)
        c_ak = c_a/((k + 1)**gamma_a)

        #Find Delta
        Delta_k = np.array(phi_k)
        for b in range(bL):
            for t in range(T):
                for q in range(Q):
                    Delta_k[b][t][q] = 1 - 2*np.random.binomial(size=None, n=1, p=0.5)
        phi_k_A = phi_k + c_k*Delta_k
        phi_k_B = phi_k - c_k*Delta_k
        
        #Find Delta Alpha
        Delta_ak = np.array(alpha_k)
        for n in range(bL):
            Delta_ak[n] = 1 - 2*np.random.binomial(size=None, n=1, p= 0.5)
        alpha_k_A = alpha_k + c_ak*Delta_ak
        alpha_k_B = alpha_k - c_ak*Delta_ak
        norm_A = 1/np.sqrt(np.dot(alpha_k_A,alpha_k_A))
        norm_B = 1/np.sqrt(np.dot(alpha_k_B,alpha_k_B))
        alpha_k_A = norm_A*alpha_k_A
        alpha_k_B = norm_B*alpha_k_B
            
        #Find E    
        user_messenger.publish({'starting iteration':k})
        E_A = find_E(backend,alpha_k_A, phi_k_A, blocks, method = method)
        user_messenger.publish({'k':k,'E_A':E_A})
        E_B = find_E(backend,alpha_k_B, phi_k_B, blocks, method = method)
        user_messenger.publish({'k':k,'E_B':E_B})
        
        #Calculate gradiant
        g = np.real((E_A-E_B)/(2*c_k)) 
        
        #Update phi
        g_k = g * Delta_k
        phi_k = phi_k - a_k * g_k
        
        #Update alpha
        g_ak = g * Delta_ak
        alpha_k = alpha_k - a_ak * g_ak
        norm = 1/np.sqrt(np.dot(alpha_k,alpha_k))
        alpha_k = norm*alpha_k

        
        #Calculate new E
        E_f = np.real(find_E(backend,alpha_k,phi_k,blocks, method = method))
        
        #Print and save E
        #print('k=',k,'c_k=',c_k,'a_k=',a_k,'g=',g,'E_A=',E_A,'E_B=',E_B,'E_f=',E_f)
        user_messenger.publish({'k':k,'E_f':E_f})
        E_l.append(E_f)
        
        if hold == True:
            hold_k = {'E':E_f,'c':c_k,'a':a_k,'phi':phi_k,'Delta':Delta_k,'c_a':c_ak,'a_a':a_ak,'alpha':alpha_k,'Delta_a':Delta_ak}
            hold_l.append(hold_k)
    if hold == True:
        return hold_l
    else:
        return E_l,phi_k



####### Main ########
def main(backend, user_messenger, **kwargs):
    k_max = kwargs.pop('k_max', 10)
    phi = kwargs.pop('phi')
    alpha = kwargs.pop('alpha')
    blocks = kwargs.pop('blocks')
    seeded = kwargs.pop('seeded',False)
    method = kwargs.pop('method','quantum')
    user_messenger.publish({"Starting program with k_max": k_max})
    out = SPSA(backend, user_messenger, k_max, phi, alpha, blocks, method = method, hold = True, seeded = seeded)
    return out



Overwriting runtime_block_vqe.py


# Upload the program

In [17]:
import json

program_details = {
  "name": "Block_VQE",
  "description": "Performs VQE on Hamiltonian blocks so that the Hamitonian can be split into classical and quantum parts.  This program was written by Dr. John Stenger.  For help email me at jstenge2@gmail.com",
  "max_execution_time": 28800,
  "version": "1.0",
  "backend_requirements": {"min_num_qubits":  3},
  "parameters": [
    {"name": "k_max", "description": "The number of iterations for the VQE algorithm", "type": "int", "required": True}
  , {"name": "phi", "description": "The starting angles for the anzats circuit.  The form must be [classical block index][circuit depth index][qubit index]", "type": "nested list of dim 3", "required": True}
  , {"name": "alpha", "description": "The starting values of the wieghts of each block.  The form must be [classical block index]", "type": "nested list of dim 3", "required": True}
  ],
  "return_values": [
    {"name": "output", "description": "A collection of paramter values at each iteration", "type": "list of dictionaries"}
  ],
  "interim_results": [
    {"name": "k", "description": "Iteration number.", "type": "int"},
    {"name": "Ef", "description": "The energy at the end of the step", "type": "int"}
  ]
}

out_file = open("runtime_block_vqe_details.json", "w")
json.dump(program_details, out_file)
out_file.close()

in_file = open("runtime_block_vqe_details.json",)
details = json.load(in_file)
in_file.close()

print(details)

{'name': 'Block_VQE', 'description': 'Performs VQE on Hamiltonian blocks so that the Hamitonian can be split into classical and quantum parts.  This program was written by Dr. John Stenger.  For help email me at jstenge2@gmail.com', 'max_execution_time': 28800, 'version': '1.0', 'backend_requirements': {'min_num_qubits': 3}, 'parameters': [{'name': 'k_max', 'description': 'The number of iterations for the VQE algorithm', 'type': 'int', 'required': True}, {'name': 'phi', 'description': 'The starting angles for the anzats circuit.  The form must be [classical block index][circuit depth index][qubit index]', 'type': 'nested list of dim 3', 'required': True}, {'name': 'alpha', 'description': 'The starting values of the wieghts of each block.  The form must be [classical block index]', 'type': 'nested list of dim 3', 'required': True}], 'return_values': [{'name': 'output', 'description': 'A collection of paramter values at each iteration', 'type': 'list of dictionaries'}], 'interim_results'

In [18]:
import os
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')  # Substitute with your provider.


ibmqfactory.load_account:WARNING:2022-01-25 08:57:53,953: Credentials are already in use. The existing account in the session will be replaced.


In [19]:
sample_program_json = os.path.join(os.getcwd(), "runtime_block_vqe_details.json")
sample_program_data = os.path.join(os.getcwd(), "runtime_block_vqe.py")

In [20]:
# This will fail if program already exists.
program_id = provider.runtime.upload_program(
    data=sample_program_data,
    metadata=sample_program_json
)
print(program_id)

QiskitRuntimeError: 'Failed to create program: \'400 Client Error: Bad Request for url: https://runtime-us-east.quantum-computing.ibm.com/programs. {"errors":[{"code":"bad_request","message":"readObjectStart: expect { or n, but found -, error found in #1 byte of ...|--fe02c9951|..., bigger context ...|--fe02c99516a0307eba0f1de13da32a9a\\\\r\\\\nContent-Disposi|...","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"c7o03q1gpneldpkjdbq0"}\''

# Delete the program

In [6]:
#Delete the program
program_id = "block-vqe"
provider.runtime.delete_program(program_id)

# Test the program

In [79]:
from qiskit.opflow import SummedOp, PauliOp
from qiskit.quantum_info import Pauli

blocks = {'0,0': [PauliOp(Pauli('III'), coeff=3.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('ZII'), coeff=3.0),
  PauliOp(Pauli('XXI'), coeff=1.0),
  PauliOp(Pauli('YYI'), coeff=2.0),
  PauliOp(Pauli('ZZI'), coeff=3.0),
  PauliOp(Pauli('IXX'), coeff=1.0),
  PauliOp(Pauli('IYY'), coeff=2.0),
  PauliOp(Pauli('IZZ'), coeff=3.0)],
 '0,1': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('XII'), coeff=1.0),
  PauliOp(Pauli('YII'), coeff=-2j),
  PauliOp(Pauli('III'), coeff=0.0)],
 '0,2': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '0,3': [PauliOp(Pauli('III'), coeff=-1.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '1,0': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('XII'), coeff=1.0),
  PauliOp(Pauli('YII'), coeff=2j),
  PauliOp(Pauli('III'), coeff=0.0)],
 '1,1': [PauliOp(Pauli('III'), coeff=-3.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('ZII'), coeff=-3.0),
  PauliOp(Pauli('XXI'), coeff=1.0),
  PauliOp(Pauli('YYI'), coeff=2.0),
  PauliOp(Pauli('ZZI'), coeff=3.0),
  PauliOp(Pauli('IXX'), coeff=1.0),
  PauliOp(Pauli('IYY'), coeff=2.0),
  PauliOp(Pauli('IZZ'), coeff=3.0)],
 '1,2': [PauliOp(Pauli('III'), coeff=3.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '1,3': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '2,0': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '2,1': [PauliOp(Pauli('III'), coeff=3.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '2,2': [PauliOp(Pauli('III'), coeff=-3.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('ZII'), coeff=3.0),
  PauliOp(Pauli('XXI'), coeff=1.0),
  PauliOp(Pauli('YYI'), coeff=2.0),
  PauliOp(Pauli('ZZI'), coeff=3.0),
  PauliOp(Pauli('IXX'), coeff=1.0),
  PauliOp(Pauli('IYY'), coeff=2.0),
  PauliOp(Pauli('IZZ'), coeff=3.0)],
 '2,3': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('XII'), coeff=1.0),
  PauliOp(Pauli('YII'), coeff=-2j),
  PauliOp(Pauli('III'), coeff=0.0)],
 '3,0': [PauliOp(Pauli('III'), coeff=-1.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '3,1': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0)],
 '3,2': [PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('XII'), coeff=1.0),
  PauliOp(Pauli('YII'), coeff=2j),
  PauliOp(Pauli('III'), coeff=0.0)],
 '3,3': [PauliOp(Pauli('III'), coeff=3.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('III'), coeff=0.0),
  PauliOp(Pauli('ZII'), coeff=-3.0),
  PauliOp(Pauli('XXI'), coeff=1.0),
  PauliOp(Pauli('YYI'), coeff=2.0),
  PauliOp(Pauli('ZZI'), coeff=3.0),
  PauliOp(Pauli('IXX'), coeff=1.0),
  PauliOp(Pauli('IYY'), coeff=2.0),
  PauliOp(Pauli('IZZ'), coeff=3.0)]}

In [80]:
bL = 4
Q = 3
alpha = 1/np.sqrt(bL)*np.array([1 for b in range(bL)])
#number of blocks
T = 2
phi = []
for b in range(bL):
    phi_b = []
    for ti in range(T):
        phi_t = []
        for q in range(Q):
            phi_t.append(0)
        phi_b.append(phi_t)
    phi.append(phi_b)

In [29]:
#Only needed if running on the quantum device
from qiskit import IBMQ,transpile
from qiskit.tools.monitor import job_monitor
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')
backend = provider.get_backend('ibmq_bogota')

ibmqfactory.load_account:WARNING:2021-10-26 11:41:56,229: Credentials are already in use. The existing account in the session will be replaced.


In [81]:
from qiskit import Aer
from qiskit.providers.ibmq.runtime import UserMessenger

backend = Aer.get_backend('qasm_simulator')
user_messenger = UserMessenger()

In [83]:
inputs = {"k_max": 5, "phi": phi, "alpha": alpha, "blocks": blocks}

out = main(backend, user_messenger, **inputs)

{"Starting program with k_max": 5}
{"k": 0, "E_f": 7.006321226979268}
{"k": 1, "E_f": 6.706742246217824}
{"k": 2, "E_f": 6.137622695025712}
{"k": 3, "E_f": 6.074707604825253}
{"k": 4, "E_f": 5.311102418276674}


In [84]:
out[4]

{'E': 5.311102418276674,
 'c': 0.33998833831625347,
 'a': 0.02901186589249766,
 'phi': array([[[ 0.01597708,  0.06947155,  0.36739054],
         [-0.16260729, -0.01597708,  0.05725458]],
 
        [[ 0.17482426, -0.05725458, -0.14752867],
         [-0.02353548, -0.22076033, -0.22076033]],
 
        [[-0.37960751, -0.02353548,  0.00376011],
         [ 0.02353548,  0.1353117 ,  0.20854336]],
 
        [[-0.1353117 ,  0.21610175, -0.22831872],
         [-0.02353548, -0.00376011, -0.37960751]]]),
 'Delta': array([[[-1.,  1.,  1.],
         [ 1.,  1.,  1.]],
 
        [[-1., -1., -1.],
         [ 1., -1., -1.]],
 
        [[-1.,  1., -1.],
         [-1.,  1.,  1.]],
 
        [[-1.,  1., -1.],
         [ 1.,  1., -1.]]]),
 'c_a': 0.33998833831625347,
 'a_a': 0.02901186589249766,
 'alpha': array([0.07887672, 0.57958964, 0.40127928, 0.70486116]),
 'Delta_a': array([-1.,  1., -1.,  1.])}